In [ ]:
import torch
import torch.nn as nn
import sys
sys.path.append('..')
import myd2l

In [2]:
def vgg_block(n_convs, in_channels, out_channels):
    layers = []
    for _ in range(n_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    
    return nn.Sequential(*layers)

In [3]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [4]:
def vgg(conv_arc):
    conv_blks = []
    in_channels = 1
    for (n_conv, out_channels) in conv_arc:
        conv_blks.append(vgg_block(n_conv, in_channels, out_channels))
        in_channels = out_channels
    
    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

In [5]:
X = torch.zeros((1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, X.shape)

Sequential torch.Size([1, 64, 112, 112])
Sequential torch.Size([1, 128, 56, 56])
Sequential torch.Size([1, 256, 28, 28])
Sequential torch.Size([1, 512, 14, 14])
Sequential torch.Size([1, 512, 7, 7])
Flatten torch.Size([1, 25088])
Linear torch.Size([1, 4096])
ReLU torch.Size([1, 4096])
Dropout torch.Size([1, 4096])
Linear torch.Size([1, 4096])
ReLU torch.Size([1, 4096])
Dropout torch.Size([1, 4096])
Linear torch.Size([1, 10])


In [ ]:
train_iter, test_iter = myd2l.load_data_fashion_mnist(batch_size=128, resize=(224, 224))

In [ ]:
lr, n_epochs = 0.05, 10
device = torch.device('cpu')

myd2l.train_clf(net, n_epochs, lr, train_iter, test_iter, device)

iter 0, loss  2.3034


KeyboardInterrupt: 